# 10 모델 검증과 튜닝

***

In [ ]:
# 데이터 가공·조작·분석 라이브러리
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series,DataFrame
import pandas as pd

# 시각화 라이브러리
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
sns.set()

# 머신러닝 라이브러리
import sklearn

# 소수점 3자리까지 표시
%precision 3

In [ ]:
# 필요한 라이브러리 임포트
from sklearn.datasets import load_breast_cancer
from sklearn.tree import  DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# 유방암 데이터 읽어 들이기
cancer = load_breast_cancer()

# 의사결정나무 클래스 초기화
tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

# k-겹 교차검증 실행
scores = cross_val_score(tree, cancer.data, cancer.target, cv=5)

# 결과 출력
print('Cross validation scores: {}'.format(scores))
print('Cross validation scores: {:.3f}+-{:.3f}'.format(scores.mean(), scores.std()))

In [ ]:
# 라이브러리 임포트
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# 유방암 데이터 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    stratify = cancer.target,
                                                    random_state=0)

# 모든 하이퍼 파라미터 조합으로 모델 구축 및 검증
scores = {}
for gamma in np.logspace(-3, 2, num=6):
    for C in np.logspace(-3, 2, num=6):
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train,y_train)
        scores[(gamma, C)] = svm.score(X_test, y_test)

# 검증 결과를 scores에 저장
scores = pd.Series(scores)

# 출력
print('최고 점수:{:.2f}'.format(scores.max()))
print('최고 점수에서의 하이퍼 파라미터(gamma, C):{}'.format(scores.idxmax()))

# 히트맵 출력, 세로축은 gamma, 가로축은 C
sns.heatmap(scores.unstack())

In [ ]:
# 임포트
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# 유방암 데이터 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    stratify = cancer.target,
                                                    random_state=0)
# GridSearchCV 클래스에 전달할 파라미터 값 지정
param_grid = { 'C': np.logspace(-3, 2, num=6)
               ,'gamma':np.logspace(-3, 2, num=6)}

# GridSearchCV 클래스 초기화
gs = GridSearchCV(estimator=SVC(),
                  param_grid=param_grid,
                  cv=5)

# 모든 하이퍼 파라미터 조합으로 모델 검증 및 베스트 모델 구축
gs.fit(X_train,y_train)

# 출력
print('Best cross validation score:{:.3f}'.format(gs.best_score_))
print('Best parameters:{}'.format(gs.best_params_))
print('Test score:{:.3f}'.format(gs.score(X_test,y_test)))

In [ ]:
# 임포트
from sklearn.svm import SVC

# 유방암 데이터 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    stratify=cancer.target,
                                                    random_state=66)
# 클래스 초기화 및 학습
model = SVC(gamma=0.001,C=1)
model.fit(X_train,y_train)

# 출력
print('{} train score: {:.3f}'.format(model.__class__.__name__, model.score(X_train,y_train)))
print('{} test score: {:.3f}'.format(model.__class__.__name__ , model.score(X_test,y_test)))

In [ ]:
# 임포트
from sklearn.metrics import confusion_matrix

# 테스트 데이터를 이용해 예측값 계산
y_pred = model.predict(X_test)

m = confusion_matrix(y_test, y_pred)
print('Confution matrix:\n{}'.format(m))

In [ ]:
accuracy = (m[0, 0] + m[1, 1]) / m.sum()
print('정확도:{:.3f}'.format(accuracy))

In [ ]:
# 정밀도 계산
precision = (m[1,1])/m[:, 1].sum()

# 재현도 계산
recall = (m[1,1])/m[1, :].sum()

# F1 점수 계산
f1 = 2 * (precision * recall)/(precision + recall)

print('정밀도:{:.3f}'.format(precision))
print('재현도:{:.3f}'.format(recall))
print('F1 값:{:.3f}'.format(f1))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print('정밀도:{:.3f}'.format(precision_score(y_test, y_pred)))
print('재현도:{:.3f}'.format(recall_score(y_test, y_pred)))
print('F1 값:{:.3f}'.format(f1_score(y_test, y_pred)))

In [ ]:
# 임포트
from sklearn.linear_model import LogisticRegression

# 유방암 데이터 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    stratify = cancer.target,
                                                    random_state=66)
# LogisticRegression 클래스 초기화 및 학습
model = LogisticRegression(random_state=0,solver='liblinear')
model.fit(X_train, y_train)

# 테스트용 데이터 예측 확률 계산
results = pd.DataFrame(model.predict_proba(X_test), columns=cancer.target_names)

# 처음 다섯 행 데이터 출력
results.head()

In [ ]:
# 양성(benign) 범주 예측 확률이 0.4, 0.3, 0.15, 0.05 일 때 각 컬럼(변수)에 1을 지정
for threshold in [0.4, 0.3, 0.15, 0.05]:
    results[f'flag_{threshold}'] = results['benign'].map(lambda x: 1 if x > threshold else 0)

# 처음 10행 데이터 출력
results.head(10)

In [ ]:
# 임곗값을 0.01~ 0.99 사이에서 50개를 만들고 위양성 비율과 진양성 비율 계산
rates = {}
for threshold in np.linspace(0.01, 0.99, num=50):
    labels = results['benign'].map(lambda x: 1 if x > threshold else 0)
    m = confusion_matrix(y_test, labels)
    rates[threshold] = {'false positive rate': m[0,1] / m[0, :].sum(),
                         'true positive rate': m[1,1] / m[1, :].sum()}
    
# 가로축은 false positive rate, 세로축은 true positive rate로 그래프 그리기
pd.DataFrame(rates).T.plot.scatter('false positive rate', 'true positive rate')

In [ ]:
# 임포트
from sklearn import svm
from sklearn.metrics import roc_curve, auc

# 유방암 데이터 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, test_size=0.5, random_state=66)

# SVC를 이용한 예측 확률 계산
model = svm.SVC(kernel='linear', probability=True, random_state=0)
model.fit(X_train, y_train)

# 예측 확률 입력
y_pred = model.predict_proba(X_test)[:,1]

In [ ]:
# 위양성 비율과 진양성 비율 계산
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# AUC 산출
auc = auc(fpr, tpr)

# ROC 곡선 그리기
plt.plot(fpr, tpr, color='red', label='ROC curve (area = %.3f)' % auc)
plt.plot([0, 1], [0, 1], color='black', linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="best")

In [ ]:
# 임포트
from sklearn.datasets import load_boston

# Housing 데이터 세트 읽어 들이기
boston = load_boston()

# DataFrame에 데이터 저장
X = pd.DataFrame(boston.data, columns=boston.feature_names)

# 주택가격 중앙값(MEDV)데이터 준비
y = pd.Series(boston.target, name='MEDV')


# X와 y를 결합하고 첫 다섯 행 출력
X.join(y).head()

In [ ]:
# 임포트
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 표준화
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

# 모델 설정
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(random_state=0),
    'DecisionTreeRegressor': DecisionTreeRegressor(random_state=0),
    'LinearSVR': LinearSVR(random_state=0)
}

# 평가 점수 계산
scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'MSE')] = mean_squared_error(y_test, model.predict(X_test))
    scores[(model_name, 'MAE')] = mean_absolute_error(y_test, model.predict(X_test))
    scores[(model_name, 'MedAE')] = median_absolute_error(y_test, model.predict(X_test))
    scores[(model_name, 'R2')] = r2_score(y_test, model.predict(X_test))

# 출력
pd.Series(scores).unstack()

In [ ]:
# 임포트
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# 유방암 데이터 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify = cancer.target, random_state=66)

# k-NN 모델과 배깅 설정
models = {
    'kNN': KNeighborsClassifier(),
    'bagging': BaggingClassifier(KNeighborsClassifier(), n_estimators=100, random_state=0) 
}

# 모델 구축
scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'train_score')] = model.score(X_train, y_train)
    scores[(model_name, 'test_score')] = model.score(X_test, y_test)

# 결과 표시
pd.Series(scores).unstack()

In [ ]:
# 임포트
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

# housing 데이터 읽어 들이기
boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(
    boston.data, boston.target, random_state=66)

# 의사결정나무와 AdaBoostRegressor 파라미터 설정
models = {
    'tree': DecisionTreeRegressor(random_state=0),
    'AdaBoost': AdaBoostRegressor(DecisionTreeRegressor(), random_state=0) 
}

# 모델 구축
scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'train_score')] = model.score(X_train, y_train)
    scores[(model_name, 'test_score')] = model.score(X_test, y_test)

# 결과 출력
pd.Series(scores).unstack()

In [ ]:
# 임포트
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Housing 데이터 읽어 들이기
boston = load_boston()

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(
    boston.data, boston.target, random_state=66)

# 랜덤 포레스트와 그레디언트 부스팅 설정
models = {
    'RandomForest': RandomForestRegressor(random_state=0),
    'GradientBoost': GradientBoostingRegressor(random_state=0) 
}

# 모델 구축
scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'train_score')] = model.score(X_train, y_train)
    scores[(model_name, 'test_score')] = model.score(X_test, y_test)

# 결과 출력
pd.Series(scores).unstack()

In [ ]:
# feature_importmnces 속성 추출
s = pd.Series(models['RandomForest'].feature_importances_,
              index=boston.feature_names)

# 추출한 값을 내림차순으로 출력
s.sort_values(ascending=False).plot.bar(color='C0')